In [19]:
import pandas as pd
import numpy as np
from PIL import Image 
import matplotlib.pyplot as plt
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
import torchvision.models as models
import torch.optim as optim
import torchvision
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import cv2
import seaborn as sns


In [20]:
Thickness = pd.read_csv('./data/Thickness.csv')
Thickness = Thickness.drop([33]) # 9-1 drop
Thickness.reset_index(drop=True, inplace=True)
Thickness

,Thickness
0,48.266667
1,47.966667
2,48.166667
3,48.600000
4,59.533333
...,...
122,84.633333
123,98.366667
124,97.033333
125,96.266667


In [21]:
img_path_list = []
Top_change_imp_VS_path = './data/box_img/Top_change_VS_imp/'

for i in range(1, 33):
    for j in range(1, 5):
        if i == 9 and j == 1: 
            continue
        img_path_list.append('./data/test_img/AI-{}_0{}_transformed.jpg'.format(i, j))

전처리한 이미지 RGB / HSV

In [22]:
top_avg_bgr = []
top_avg_hsv = []

for i in range(len(img_path_list)):
    # 이미지 로드
    img = cv2.cvtColor(np.array(Image.open(Top_change_imp_VS_path + 'Top_change_VS_{}.jpg'.format(i))), cv2.COLOR_RGB2BGR)
    # RGB 평균 계산
    avg_bgr = img.mean(axis=(0,1))
    top_avg_bgr.append(avg_bgr)
    # HSV 평균 계산
    img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    avg_hsv = img_hsv.mean(axis=(0,1))
    top_avg_hsv.append(avg_hsv)

In [28]:
Table_dataset = pd.DataFrame({'Top_R' : [i[2] for i in top_avg_bgr], 
                              'Top_G' : [i[1] for i in top_avg_bgr], 
                              'Top_B' : [i[0] for i in top_avg_bgr],
                              'Top_H' : [i[0] for i in top_avg_hsv], 
                              'Top_S' : [i[1] for i in top_avg_hsv], 
                              'Top_V' : [i[2] for i in top_avg_hsv],
                              'Thickness': Thickness['Thickness']})

Table_dataset[0:20]

,Top_R,Top_G,Top_B,Top_H,Top_S,Top_V,Thickness
0,11.5752,10.8824,8.0332,22.1580,79.0864,11.5752,48.266667
1,10.2804,10.4340,9.0232,23.5584,41.5664,10.5508,47.966667
2,10.0228,9.9236,7.9204,29.0352,56.4348,10.0308,48.166667
3,8.3744,8.5136,6.1348,26.7236,77.8136,8.6560,48.600000
4,4.1716,2.2884,1.1488,27.9292,211.7900,4.3000,59.533333
5,3.6796,3.3596,1.2708,26.7248,180.2076,3.6796,59.766667
6,7.4280,5.1608,3.1944,15.0744,154.8588,7.4280,59.800000
7,14.6844,4.9792,35.1944,65.7416,223.3360,40.5384,60.100000
8,1.3584,1.0224,1.0276,29.6360,107.0820,1.6552,68.266667
9,0.6500,0.5644,1.6964,66.8984,125.7508,1.7092,68.833333


원본데이터

In [24]:
Top_path = './data/box_img/Top/'

top_org_avg_bgr = []
top_org_avg_hsv = []

for i in range(len(img_path_list)):
    # 이미지 로드
    img = cv2.cvtColor(np.array(Image.open(Top_path + 'Top_{}.jpg'.format(i))), cv2.COLOR_RGB2BGR)
    # RGB 평균 계산
    avg_bgr = img.mean(axis=(0,1))
    top_org_avg_bgr.append(avg_bgr)
    # HSV 평균 계산
    img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    avg_hsv = img_hsv.mean(axis=(0,1))
    top_org_avg_hsv.append(avg_hsv)


In [29]:
orig_Table_dataset = pd.DataFrame({'Top_R' : [i[2] for i in top_org_avg_bgr], 
                              'Top_G' : [i[1] for i in top_org_avg_bgr], 
                              'Top_B' : [i[0] for i in top_org_avg_bgr],
                              'Top_H' : [i[0] for i in top_org_avg_hsv], 
                              'Top_S' : [i[1] for i in top_org_avg_hsv], 
                              'Top_V' : [i[2] for i in top_org_avg_hsv],
                              'Thickness': Thickness['Thickness']})

orig_Table_dataset[0:20]

,Top_R,Top_G,Top_B,Top_H,Top_S,Top_V,Thickness
0,11.5760,10.8832,8.1364,22.0556,77.1296,11.5760,48.266667
1,12.4712,12.6248,11.2136,23.5584,34.0868,12.7416,47.966667
2,12.7704,12.7176,9.9168,29.7648,60.9592,12.9592,48.166667
3,11.2308,11.4564,8.8540,26.6792,61.8904,11.6148,48.600000
4,5.4696,3.2120,1.4352,30.2408,205.4796,5.5860,59.533333
5,5.7508,5.2796,0.8820,29.3668,224.3508,6.4204,59.766667
6,10.8848,4.1844,1.0452,9.9820,232.9496,10.8848,59.800000
7,3.0396,5.3108,1.5084,46.8728,194.0972,5.3620,60.100000
8,2.5236,1.8392,1.8368,43.2324,142.5788,2.9608,68.266667
9,1.3556,1.3124,3.3556,101.1688,167.4976,3.4836,68.833333


안쓸데이터 드랍

In [26]:
Table_dataset = Table_dataset.drop(['Top_H'], axis=1)
Table_dataset

,Top_R,Top_G,Top_B,Top_S,Top_V,Thickness
0,11.5752,10.8824,8.0332,79.0864,11.5752,48.266667
1,10.2804,10.4340,9.0232,41.5664,10.5508,47.966667
2,10.0228,9.9236,7.9204,56.4348,10.0308,48.166667
3,8.3744,8.5136,6.1348,77.8136,8.6560,48.600000
4,4.1716,2.2884,1.1488,211.7900,4.3000,59.533333
...,...,...,...,...,...,...
122,1.8884,5.0128,18.5152,229.4796,18.5152,84.633333
123,6.9784,15.3760,26.5632,188.4016,26.5632,98.366667
124,3.3400,13.3692,33.3720,230.0004,33.3720,97.033333
125,5.8372,15.0308,31.7036,208.5904,31.7036,96.266667


데이터 나누기

In [27]:
X_train, X_test, y_train, y_test = train_test_split(Table_dataset.iloc[:, :-1], Table_dataset.iloc[:, -1], test_size=0.3, random_state=42)